In [ ]:
import ossdbs
from netgen.webgui import Draw
from ngsolve import TaskManager
from ngsolve.webgui import Draw as DrawNG

# Settings

In [ ]:
settings = { 
  "Electrodes":
  [
    {
      "Name": "BostonScientificVercise",
      "PathToCustomParameters": "",
      "Rotation[Degrees]": 0.0,
      "Direction": { "x[mm]": 0.0, "y[mm]": 0.0, "z[mm]": 1.0 },
      "TipPosition": { "x[mm]": 17.0, "y[mm]": 8.0, "z[mm]": 6.0 },

      "Contacts": [
        {
          "Contact_ID": 1,
          "Active": True,
          "Current[A]": 0.0,
          "Voltage[V]": 1.0,
          "Floating": False,
          "SurfaceImpedance[Ohmm]": { "real": 0.0, "imag": 0.0 }
        },
        {
          "Contact_ID": 2,
          "Active": True,
          "Current[A]": 0.0,
          "Voltage[V]": 0.0,
          "Floating": False,
          "SurfaceImpedance[Ohmm]": { "real": 0.0, "imag": 0.0 }
        }
      ],
    "EncapsulationLayer": {
        "Thickness[mm]": 0.0,
        "Material": "Blood",
        "DielectricModel": "ColeCole4",
        "MaxMeshSize": 0.5
     }
    }
  ],

  "BrainRegion": {
    "Center": { "x[mm]": 17.0, "y[mm]": 8.0, "z[mm]": 6.0 },
    "Dimension": { "x[mm]": 50.0, "y[mm]": 50.0, "z[mm]": 50.0 }
  },

  "MaterialDistribution": {
    "MRIPath": "../input_files/icbm_avg_152_segmented.nii.gz",
    "MRIMapping": {
      "Unknown": 0,
      "CSF": 1,
      "White matter": 2,
      "Gray matter": 3,
      "Blood": 4      
    },
    "DiffusionTensorActive": False,
    "DTIPath": ""
  },

  "DielectricModel": {
    "Type": "ColeCole4"
  },

  "Mesh": {
    "LoadMesh": False,
    "MeshElementOrder": 2,
    "MeshingHypothesis": {
      "Type": "Default",
      "MaxMeshSize": 0.1
    },
    "MeshSize":{
      "Edges": {},
      "Faces":{"E1C1": 0.1},
      "Volumes":{"Brain": 0.5}
    },
    "SaveMesh": False
  },

  "MeshRefinement": {
    "MeshSizeRatioContacts": 3.0
  },

  "FEMOrder": 2,
  "EQSMode": False, 
  "ComputeImpedance": True,

  "StimulationSignal": {
    "CurrentControlled": False,
    "Type": "Multisine",
    "ListOfFrequencies": [130.0]
  },
    
  "Solver": {
    "Type": "CG",
    "Preconditioner": "bddc",
    "PrintRates": False,
    "MaximumSteps": 10000,
    "Precision": 1e-12
  },

  "OutputPath": "",
  "SaveImpedance": True,
  "ExportVTK": True
}


# MRI and Geometry

In [ ]:
mri_image = ossdbs.MagneticResonanceImage(settings['MaterialDistribution']['MRIPath'])

In [ ]:
electrodes = ossdbs.generate_electrodes(settings)
electrode = electrodes[0]
Draw(electrode.geometry)

In [ ]:
bounding_box = ossdbs.create_bounding_box(settings["BrainRegion"])
brain = ossdbs.BrainGeometry("Ellipsoid", bounding_box)

In [ ]:
model_geometry = ossdbs.ModelGeometry(brain, electrodes)
Draw(model_geometry.geometry.shape)

# Prepare VCM

In [ ]:
ossdbs.set_contact_and_encapsulation_layer_properties(settings, model_geometry)

In [ ]:
dielectric_model = ossdbs.prepare_dielectric_properties(settings)

In [ ]:
materials = settings["MaterialDistribution"]["MRIMapping"]
conductivity = ossdbs.ConductivityCF(mri_image,
                                  bounding_box,
                                  dielectric_model,
                                  materials,
                                  model_geometry.encapsulation_layers,
                                  complex_data=settings["EQSMode"]
                                  )

In [ ]:
solver = ossdbs.prepare_solver(settings)

In [ ]:
with TaskManager():
    volume_conductor = ossdbs.prepare_volume_conductor_model(settings, model_geometry, conductivity, solver)

# Run VCM

In [ ]:
with TaskManager():
    ossdbs.run_volume_conductor_model(settings, volume_conductor)

In [ ]:
DrawNG(volume_conductor.potential)

In [ ]:
import pickle
file_out = "potential.data"
pickler = pickle.Pickler(open(file_out,"wb"))
data = [volume_conductor.potential, volume_conductor.mesh]
pickler.dump(data)